In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
import sqlite3
import re

**Парсинг сайта и настройка базы данных**

**1. Парсинг**

In [3]:
urls = ["https://text-you.ru/songs/григорий+лепс.html", "https://text-you.ru/songs/григорий+лепс-2.html", "https://text-you.ru/songs/григорий+лепс-3.html"] # ссылки на исполнителей

songs_lists = [] # создадим пустой список для добавления в них ссылок с текстами песен

for url in urls: # вытащим из html адреса с текстами и добавим их в список
    songs_list = requests.get(url)
    songs_soup = BeautifulSoup(songs_list.content, "html.parser")
    for a in songs_soup.find_all('a'):
        if str(a).find('rus_text_pesni') != -1 and str(a).find('leps') != -1:
            song_url = 'https://text-you.ru/' + a['href']
            songs_lists.append(song_url)

songs_lists = list(set(songs_lists))  # убираем дубликаты
songs_lists

['https://text-you.ru//rus_text_pesni/67188-grigorijj-leps-ne-tronte-dushu-grjaznymi-rukami.html',
 'https://text-you.ru//rus_text_pesni/4565-grigorijj-leps-kupola.html',
 'https://text-you.ru//rus_text_pesni/4570-grigorijj-leps-izmeny-i-viktorija-ilinskaja.html',
 'https://text-you.ru//rus_text_pesni/4559-grigorijj-leps-vjuga.html',
 'https://text-you.ru//rus_text_pesni/4557-grigorijj-leps-natali.html',
 'https://text-you.ru//rus_text_pesni/10160-grigorijj-leps-zamerzaet-solnce.html',
 'https://text-you.ru//rus_text_pesni/11733-grigorijj-leps-zima.html',
 'https://text-you.ru//rus_text_pesni/65632-grigorijj-leps-dom-khrustalnyjj.html',
 'https://text-you.ru//rus_text_pesni/70691-grigorijj-leps-ona-menja-baluet.html',
 'https://text-you.ru//rus_text_pesni/4563-grigorijj-leps-berega.html',
 'https://text-you.ru//rus_text_pesni/4558-grigorijj-leps-ja-tebja-ne-ljublju.html',
 'https://text-you.ru//rus_text_pesni/4571-grigorijj-leps-ty-opozdala.html',
 'https://text-you.ru//rus_text_pesni/

In [4]:
# функция очистки текста от лишних символов
def replace_all(text, to_replace):
    for i in to_replace:
        text = text.replace(i, ' ')
    return text

In [5]:
parsed_songs = {}
n = 0 # переменная для подсчёта количества песен

for song_name in songs_lists:
  song_url = requests.get(song_name)
  song_soup = BeautifulSoup(song_url.content, "html.parser")
  song_text = str(song_soup.find_all('pre')[0]) # находим нужную часть с текстом
  song_text = replace_all(song_text, ['<pre>', '</pre>', '<br/>']) # очищаем тексты от лишних символов
  song_text = song_text.rstrip().lstrip()  # убираем пробелы в начале и в конце
  parsed_songs[song_name] = song_text
  n += 1
  print("Parsed", n, "/", len(songs_lists))
  time.sleep(1)

Parsed 1 / 56
Parsed 2 / 56
Parsed 3 / 56
Parsed 4 / 56
Parsed 5 / 56
Parsed 6 / 56
Parsed 7 / 56
Parsed 8 / 56
Parsed 9 / 56
Parsed 10 / 56
Parsed 11 / 56
Parsed 12 / 56
Parsed 13 / 56
Parsed 14 / 56
Parsed 15 / 56
Parsed 16 / 56
Parsed 17 / 56
Parsed 18 / 56
Parsed 19 / 56
Parsed 20 / 56
Parsed 21 / 56
Parsed 22 / 56
Parsed 23 / 56
Parsed 24 / 56
Parsed 25 / 56
Parsed 26 / 56
Parsed 27 / 56
Parsed 28 / 56
Parsed 29 / 56
Parsed 30 / 56
Parsed 31 / 56
Parsed 32 / 56
Parsed 33 / 56
Parsed 34 / 56
Parsed 35 / 56
Parsed 36 / 56
Parsed 37 / 56
Parsed 38 / 56
Parsed 39 / 56
Parsed 40 / 56
Parsed 41 / 56
Parsed 42 / 56
Parsed 43 / 56
Parsed 44 / 56
Parsed 45 / 56
Parsed 46 / 56
Parsed 47 / 56
Parsed 48 / 56
Parsed 49 / 56
Parsed 50 / 56
Parsed 51 / 56
Parsed 52 / 56
Parsed 53 / 56
Parsed 54 / 56
Parsed 55 / 56
Parsed 56 / 56


**2. Работа с БД**

In [10]:
con = sqlite3.connect('songs.db') # создаем локальную базу данных

cur = con.cursor()
cur.execute("CREATE TABLE text_songs(author, text, url)") # создаем таблицу


for k, v in parsed_songs.items():  # перебираем ключи и значения и записываем их в таблицу
  sql_insert = """
  INSERT INTO text_songs (author, text, url)
  VALUES ('{}', '{}', '{}');
  """.format('Григорий Лепс', v, k)

  res = cur.execute(sql_insert)
  con.commit()


res = cur.execute("Select * From text_songs")
result = res.fetchall()

print(len(result), result[0])

56 ('Григорий Лепс', '[Куплет 1]: Пред тем, как я предстану идиотом, Я попрошу пред образом: "Подайте..." При нём не надо думать об исходе, Он любит нас — добавит день к расплате.  Из ничего в ничто бросая семя, Живу себе, себя воображая, Моя надежда сильно поседела И с каждым годом просто никакая.  [Припев]: Не троньте душу грязными руками! Не надо врать, что нет пути другого. Я на задворках мёртвыми губами… Искал поддержки я у чужих пророков.  [Куплет 2]: Сквозь время память обладает силой. Забыв про боль обид и гиблые трясины, Стучится в сердце птицей синекрылой, Твердит, что верность и любовь едины.   [Припев]: Не троньте душу грязными руками! Не надо врать, что нет пути другого. Я на задворках мёртвыми губами… Искал поддержки я у чужих пророков.  [Припев]: Не троньте душу грязными руками! Не надо врать, что нет пути другого. Я на задворках мёртвыми губами… Искал поддержки я у чужих пророков.  Я на задворках мёртвыми губами… Искал поддержки я у чужих пророков. У чужих пророков. У ч

In [13]:
text_prep_all = []
to_replace = ['григорий', 'лепс', '!', '?', '.', ',', ':', '-', '  ', '[', ']', 'ла', '))', '"', '(', ')', '—', '  ', 'не'] # список с символами для удаления из текстов
text_songs = [i[1] for i in result]


for i in text_songs:
    text_prep = re.findall('[А-Я][^А-Я]*', i) # делим текст на строки
    text_prep = [a.lower() for a in text_prep] # убираем заглавные буквы
    text_prep = [replace_all(a, to_replace) for a in text_prep] # очищаем тексты от лишних символов
    text_prep = [a.rstrip().rstrip() for a in text_prep] # удаляем лишние пробелы
    text_prep = [a.lstrip() for a in text_prep]
    clean_text = []
    for idx, val in enumerate(text_prep):
        if text_prep[idx] in [' ', '']: # проверяем пустая строка или нет
            pass
        else:
            clean_text.append(val) # если строка не пустая, то добавляем в clean_text
            
    text_prep = []
    for val in clean_text: # убираем короткие фразы, т.к. модель плохо обучается на коротких фразах
        if len(val) <= 10:
            pass
        else:
            text_prep.append(val)
    
    text_prep_unique = list(set(text_prep))
    text_prep_str = '\n'.join(text_prep_unique) # добавим /n, т.к. модель ожидает каждую фразу с новой строчки
    text_prep_all.append(text_prep_str)
    
print(text_prep_all[0])  

забыв про боль обид и гиблые трясины
он любит нас  добавит день к расп те
при нём   надо думать об исходе
живу себе себя воображая
моя надежда сильно поседе
и с каждым годом просто никакая
я на задворках мёртвыми губами…
стучится в сердце птицей си крылой
пред тем как я предстану идиотом
из ничего в ничто бросая семя
твердит что верность и любовь едины
троньте душу грязными руками
сквозь время память об дает силой
я попрошу пред образом
надо врать что  т пути другого
искал поддержки я у чужих пророков
у чужих пророков


In [ ]:
# создадим датафрейм и сохраним в csv
df_prepped = pd.DataFrame()


df_prepped['text'] = text_prep_all * 10
df_prepped['artist'] = 'Григорий Лепс'
df_prepped = df_prepped[['artist', 'text']]
df_prepped.to_csv('git_new/lyrics-generator/leps_text.csv', index=False)


print(df_prepped.shape)
df_prepped.head()

(560, 2)


,artist,text
0,Григорий Лепс,времени в разбитой чашке больше т\nно любовь ...
1,Григорий Лепс,я буду жить тобою\nкто же я без тебя видно уже...
2,Григорий Лепс,да в этот раз я потерпел фиаско\nи маскарад за...
3,Григорий Лепс,но когда весь мир надоест то есть куда бежать\...
4,Григорий Лепс,нарисуй луну оставь её\nсметает лив м мои след...
